In [ ]:
!pip install -q streamlit transformers sentence-transformers faiss-cpu bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [ ]:
pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.5 MB/s eta 0:00:00


In [ ]:
import zipfile
from google.colab import files

with zipfile.ZipFile("/content/kcc_chatbot_gemma.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")
with zipfile.ZipFile("/content/saved_embeddings (1).zip", "r") as zip_ref:
    zip_ref.extractall("dataset")

In [ ]:
%%writefile app.py
from transformers import AutoTokenizer, AutoModelForCausalLM
import faiss
import pickle
import streamlit as st

model = AutoModelForCausalLM.from_pretrained("/content/dataset/kcc_chatbot_gemma", trust_remote_code=True).to("cuda")

tokenizer = AutoTokenizer.from_pretrained("/content/dataset/kcc_chatbot_gemma", trust_remote_code=True)

index = faiss.read_index("/content/dataset/faiss_index.index")

with open("/content/dataset/chunks.pkl", "rb") as f:
    chunks = pickle.load(f)

import torch
import numpy as np
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


def get_context(query, top_k=3):
    with torch.no_grad():
        query_embedding = embedding_model.encode([query], convert_to_tensor=True)[0]
    D, I = index.search(np.array([query_embedding.cpu().numpy()], dtype=np.float32), top_k)
    return "\n".join([chunks[i] for i in I[0]])

from duckduckgo_search import DDGS

def internet_search(query, max_results=3):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=max_results)
        return "\n".join([result["body"] for result in results if "body" in result])


def get_context(query, top_k=3, relevance_threshold=0.6):
    with torch.no_grad():
        try:
            query_embedding = embedding_model.encode([query], convert_to_tensor=True)[0]
        except RuntimeError as e:
            print(f"Error embedding query: '{query}'")
            raise e

    D, I = index.search(np.array([query_embedding.cpu().numpy()], dtype=np.float32), top_k)

    if all(distance > relevance_threshold for distance in D[0]):
        return None

    return "\n".join([chunks[i] for i in I[0]])


def generate_answer(query):
    context = get_context(query,5)

    if context is None:
      st.markdown("No local context found, using internet fallback to generate answer.")
      context = internet_search(query)
    prompt = f"""Answer the question as from the provided context, make sure to provide all the methods/points, don't provide the wrong answer and don't repeat yourself, do no repeat the same text to fill the token size. If text is not enought for max token size don't generate stuff just leave as is. If using internet fallback, understand and analyse the search result and return an answer which makes sense. \n\n
    Context:
    {context}

    Question: {query}
    Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.8,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()




# query = "What issues do sugarcane farmers in Maharashtra commonly face?"
# print(generate_answer(query))



st.title("Agricultural Help ChatBot")

query = st.text_input("Ask a question:")
if query:
    with st.spinner("Generating answer..."):
        answer = generate_answer(query)
        st.markdown(f"**Answer:** {answer}")

Overwriting app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 924ms
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.87.46.90
⠙your url is: https://sad-bags-nail.loca.lt
